In [16]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [17]:
# ============================================================
# Configuration
# ============================================================

HIDDEN_DIM = 16     # start small, scale to 32 later
BATCH_SIZE = 64
EPOCHS = 40
LR = 1e-3
SEED = 123
EXPORT_DIR = "export"

Q_FRAC = 16
Q_SCALE = 1 << Q_FRAC

torch.manual_seed(SEED)
np.random.seed(SEED)

# ============================================================
# Load Dataset
# ============================================================

digits = load_digits()
X = digits.data.astype(np.float32) / 16.0   # normalize to [0,1]
y = digits.target.astype(np.int64)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_test  = torch.tensor(X_test)
y_test  = torch.tensor(y_test)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True
)

# ============================================================
# Model Definition
# ============================================================

class TinyMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(64, HIDDEN_DIM)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(HIDDEN_DIM, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  # raw logits (no softmax)

model = TinyMLP()

# ============================================================
# Training
# ============================================================

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

print("Training...")

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1:02d} | Loss: {total_loss:.4f}")

# ============================================================
# Evaluation
# ============================================================

model.eval()
with torch.no_grad():
    logits = model(X_test)
    preds = torch.argmax(logits, dim=1)

acc = accuracy_score(y_test.numpy(), preds.numpy())
print(f"\nTest Accuracy: {acc:.4f}")



Training...
Epoch 01 | Loss: 52.8180
Epoch 02 | Loss: 50.9024
Epoch 03 | Loss: 48.5352
Epoch 04 | Loss: 45.8121
Epoch 05 | Loss: 42.7972
Epoch 06 | Loss: 39.4486
Epoch 07 | Loss: 36.0260
Epoch 08 | Loss: 32.4281
Epoch 09 | Loss: 28.7962
Epoch 10 | Loss: 25.5361
Epoch 11 | Loss: 22.8034
Epoch 12 | Loss: 20.3684
Epoch 13 | Loss: 18.3227
Epoch 14 | Loss: 16.6349
Epoch 15 | Loss: 15.2956
Epoch 16 | Loss: 13.9969
Epoch 17 | Loss: 13.0208
Epoch 18 | Loss: 12.1856
Epoch 19 | Loss: 11.2837
Epoch 20 | Loss: 10.6244
Epoch 21 | Loss: 9.9670
Epoch 22 | Loss: 9.4425
Epoch 23 | Loss: 8.8868
Epoch 24 | Loss: 8.4056
Epoch 25 | Loss: 8.0549
Epoch 26 | Loss: 7.6204
Epoch 27 | Loss: 7.4250
Epoch 28 | Loss: 7.1450
Epoch 29 | Loss: 6.8520
Epoch 30 | Loss: 6.5092
Epoch 31 | Loss: 6.2853
Epoch 32 | Loss: 6.0484
Epoch 33 | Loss: 5.8125
Epoch 34 | Loss: 5.5902
Epoch 35 | Loss: 5.4001
Epoch 36 | Loss: 5.2923
Epoch 37 | Loss: 5.1535
Epoch 38 | Loss: 5.0142
Epoch 39 | Loss: 4.9102
Epoch 40 | Loss: 4.7302

Test Ac

In [18]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ============================================================
# Configuration
# ============================================================

HIDDEN_DIM = 16     # start small, scale to 32 later
BATCH_SIZE = 64
EPOCHS = 40
LR = 1e-3
SEED = 123
EXPORT_DIR = "export"

Q_FRAC = 16
Q_SCALE = 1 << Q_FRAC

torch.manual_seed(SEED)
np.random.seed(SEED)

# ============================================================
# Load Dataset
# ============================================================

digits = load_digits()
X = digits.data.astype(np.float32) / 16.0   # normalize to [0,1]
y = digits.target.astype(np.int64)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_test  = torch.tensor(X_test)
y_test  = torch.tensor(y_test)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True
)

# ============================================================
# Model Definition
# ============================================================

class TinyMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(64, HIDDEN_DIM)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(HIDDEN_DIM, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  # raw logits (no softmax)

model = TinyMLP()

# ============================================================
# Training
# ============================================================

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

print("Training...")

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1:02d} | Loss: {total_loss:.4f}")

# ============================================================
# Evaluation
# ============================================================

model.eval()
with torch.no_grad():
    logits = model(X_test)
    preds = torch.argmax(logits, dim=1)

acc = accuracy_score(y_test.numpy(), preds.numpy())
print(f"\nTest Accuracy: {acc:.4f}")

# ============================================================
# Extract Weights
# ============================================================

W1 = model.fc1.weight.detach().numpy()     # (HIDDEN_DIM, 64)
b1 = model.fc1.bias.detach().numpy()       # (HIDDEN_DIM)
W2 = model.fc2.weight.detach().numpy()     # (10, HIDDEN_DIM)
b2 = model.fc2.bias.detach().numpy()       # (10)

# ============================================================
# Quantization to Q16.16
# ============================================================

def float_to_q16(x):
    q = np.round(x * Q_SCALE)
    q = np.clip(q, -2**31, 2**31 - 1)
    return q.astype(np.int32)

W1_q = float_to_q16(W1)
b1_q = float_to_q16(b1)
W2_q = float_to_q16(W2)
b2_q = float_to_q16(b2)

# ============================================================
# Export Header File
# ============================================================

os.makedirs(EXPORT_DIR, exist_ok=True)

header_path = os.path.join(EXPORT_DIR, "mlp_digits_q16_16.h")

with open(header_path, "w") as f:
    f.write("// Auto-generated PyTorch Q16.16 MLP weights\n\n")
    f.write("#pragma once\n\n")
    f.write("#include <stdint.h>\n\n")
    f.write(f"#define MLP_IN_DIM 64\n")
    f.write(f"#define MLP_HID_DIM {HIDDEN_DIM}\n")
    f.write(f"#define MLP_OUT_DIM 10\n")
    f.write(f"#define MLP_Q_FRAC {Q_FRAC}\n\n")

    def write_array(name, arr):
        flat = arr.flatten()
        f.write(f"static const int32_t {name}[{len(flat)}] = {{\n    ")
        for i, val in enumerate(flat):
            f.write(str(val))
            if i != len(flat) - 1:
                f.write(", ")
            if (i+1) % 8 == 0:
                f.write("\n    ")
        f.write("\n};\n\n")

    write_array("MLP_W1_Q16", W1_q)
    write_array("MLP_B1_Q16", b1_q)
    write_array("MLP_W2_Q16", W2_q)
    write_array("MLP_B2_Q16", b2_q)

print(f"\nExported weights to {header_path}")


Training...
Epoch 01 | Loss: 52.8180
Epoch 02 | Loss: 50.9024
Epoch 03 | Loss: 48.5352
Epoch 04 | Loss: 45.8121
Epoch 05 | Loss: 42.7972
Epoch 06 | Loss: 39.4486
Epoch 07 | Loss: 36.0260
Epoch 08 | Loss: 32.4281
Epoch 09 | Loss: 28.7962
Epoch 10 | Loss: 25.5361
Epoch 11 | Loss: 22.8034
Epoch 12 | Loss: 20.3684
Epoch 13 | Loss: 18.3227
Epoch 14 | Loss: 16.6349
Epoch 15 | Loss: 15.2956
Epoch 16 | Loss: 13.9969
Epoch 17 | Loss: 13.0208
Epoch 18 | Loss: 12.1856
Epoch 19 | Loss: 11.2837
Epoch 20 | Loss: 10.6244
Epoch 21 | Loss: 9.9670
Epoch 22 | Loss: 9.4425
Epoch 23 | Loss: 8.8868
Epoch 24 | Loss: 8.4056
Epoch 25 | Loss: 8.0549
Epoch 26 | Loss: 7.6204
Epoch 27 | Loss: 7.4250
Epoch 28 | Loss: 7.1450
Epoch 29 | Loss: 6.8520
Epoch 30 | Loss: 6.5092
Epoch 31 | Loss: 6.2853
Epoch 32 | Loss: 6.0484
Epoch 33 | Loss: 5.8125
Epoch 34 | Loss: 5.5902
Epoch 35 | Loss: 5.4001
Epoch 36 | Loss: 5.2923
Epoch 37 | Loss: 5.1535
Epoch 38 | Loss: 5.0142
Epoch 39 | Loss: 4.9102
Epoch 40 | Loss: 4.7302

Test Ac